# Quantum PR

The quantum-corrected Peng-Robinson model of Aasen *et al.* (https://doi.org/10.1063/1.5111364) can be used to account for quantum effects by empirical fits to the Feynman-Hibbs corrections.

The conventional Peng-Robinson approach is used, with an adjusted covolume b_i given by

$$
b_i = b_{i, PR}\beta_i(T)
$$
with 
$$
\beta_i(T) = \left(\frac{1+A_i/(T+B_i)}{1+A_i/(T_{ci} + B_i)}\right)^3
$$
and Twu alpha functions are used to correct the attractive part.

In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas
import CoolProp.CoolProp as CP

import teqp
teqp.__version__

In [ ]:
cfactor = 0

jH2Ne = {
    "kind": "QCPRAasen",
    "model": {
        "Ls": [156.21, 0.40453],
        "Ms": [-0.0062072, 0.95861],
        "Ns": [5.047, 0.8396],
        "As": [3.0696, 0.4673],
        "Bs": [12.682, 2.4634],
        "cs": [cfactor*3.8139e-6, cfactor*2.4665e-6],
        "Tcrit / K": [33.19, 44.492],
        "pcrit / Pa": [12.964e5, 26.79e5],
        "kmat": [[0.0, 0.18], [0.18, 0.0]],
        "lmat": [[0.0, 0.0], [0.0, 0.0]]
    }
}

jNe = {
    "kind": "QCPRAasen",
    "model": {
        "Ls": [0.40453],
        "Ms": [0.95861],
        "Ns": [0.8396],
        "As": [0.4673],
        "Bs": [2.4634],
        "cs": [cfactor*2.4665e-6],
        "Tcrit / K": [44.492],
        "pcrit / Pa": [26.79e5],
        "kmat": [[0.0]],
        "lmat": [[0.0]]
    }
}

jH2 = {
    "kind": "QCPRAasen",
    "model": {
        "Ls": [156.21],
        "Ms": [-0.0062072],
        "Ns": [5.047],
        "As": [3.0696],
        "Bs": [12.682],
        "cs": [cfactor*3.8139e-6],
        "Tcrit / K": [33.19],
        "pcrit / Pa": [12.964e5],
        "kmat": [[0.0]],
        "lmat": [[0.0]]
    }
}

model = teqp.make_model(jH2Ne)
modelH2 = teqp.make_model(jH2)
modelNe = teqp.make_model(jNe)

def get_traces(T, ipures):
    traces = []
    for ipure in ipures:
        rhovecL0 = np.array([0.0, 0.0])
        rhovecV0 = np.array([0.0, 0.0])
        if ipure == 1:            
            rhoL, rhoV = modelNe.superanc_rhoLV(T)
        else:
            rhoL, rhoV = modelH2.superanc_rhoLV(T)
        rhovecL0[ipure] = rhoL
        rhovecV0[ipure] = rhoV

        opt = teqp.TVLEOptions(); 
#         opt.polish=True; 
#         opt.integration_order=5; opt.rel_err=1e-10; 
#         opt.calc_criticality = True; 
        opt.crit_termination=1e-10
        trace = model.trace_VLE_isotherm_binary(T, rhovecL0, rhovecV0, opt)
        traces.append(trace)
    return traces

for T in [24.59, 28.0, 34.66, 39.57, 42.50]:
    if T < 26.0:
        traces = get_traces(T, [0, 1])
    else:
        traces = get_traces(T, [1])

    for trace in traces:
        df = pandas.DataFrame(trace)
        
        # Plot the VLE solution
        line, = plt.plot(df['xL_0 / mole frac.'], df['pL / Pa']/1e5)
        plt.plot(df['xV_0 / mole frac.'], df['pL / Pa']/1e5, color=line.get_color())

    # Plot the VLLE solution if found
    for soln in model.find_VLLE_T_binary(traces):
        for rhovec in soln['polished']:
            rhovec = np.array(rhovec)
            rhotot = sum(rhovec)
            x = rhovec/rhotot
            p = rhotot*model.get_R(x)*T*(1+model.get_Ar01(T, rhotot, x))
            plt.plot(x[0], p/1e5, 'X', color=line.get_color())
            print(T, rhovec, x[0], p/1e5, 'bar')

plt.gca().set(xlabel='x/y H$_2$', ylabel='$P$ / bar', xlim=(0,1), ylim=(0,30));